In [1]:
import torch
from tvm import relay
import tvm
from collections import namedtuple


In [2]:
Workload = namedtuple(
    "Conv2DWorkload",
    [
        "batch",
        "height",
        "width",
        "in_filter",
        "out_filter",
        "hkernel",
        "wkernel",
        "hpad",
        "wpad",
        "hstride",
        "wstride",
    ],
)

In [28]:
pytorch_model = torch.hub.load('pytorch/vision', "googlenet", pretrained=True)


Using cache found in /home/srchand/.cache/torch/hub/pytorch_vision_main
/home/srchand/.cache/torch/hub/pytorch_vision_main/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [29]:
workloads = []
for layer in pytorch_model.modules():
    if type(layer) == torch.nn.modules.pooling.MaxPool2d:
        print(layer)
#         if(layer.in_channels % 16 == 0 and layer.out_channels % 16 ==0):
#             workloads.append(Workload(1, 0, 0, layer.in_channels, layer.out_channels,
#                               layer.kernel_size[0], layer.kernel_size[1], layer.padding[0], layer.padding[1]
#                              , layer.stride[0], layer.stride[1]))

MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
MaxPool2d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=True)
MaxPool2d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=True)
MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
MaxPool2d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=True)
MaxPool2d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=True)
MaxPool2d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=True)
MaxPool2d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=True)
MaxPool2d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=True)
MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
MaxPool2d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=True)
MaxPool2d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=True)


In [30]:
input_shape = [1, 3, 224, 224]
input_data = torch.randn(input_shape)
scripted_model = torch.jit.trace(pytorch_model, input_data).eval()

/home/srchand/miniconda3/envs/tvm-build-il-2/lib/python3.8/site-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"
/home/srchand/miniconda3/envs/tvm-build-il-2/lib/python3.8/site-packages/torch/jit/_trace.py:983: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1000 / 1000 (100.0%)
Greatest absolute difference: 1.643597960472107 at index (0, 398) (up to 1e-05 allowed)
Greatest relative difference: 814.5113282402701 at index (0, 352) (up to 1e-05 allowed)

  _check_trace(


In [31]:
shape_list = [("input0", input_shape)]
mod, params = relay.frontend.from_pytorch(scripted_model, shape_list)

In [32]:
with tvm.transform.PassContext(opt_level=3):
    with relay.quantize.qconfig(global_scale=8.0, skip_conv_layers=[0]):
         mod = relay.quantize.quantize(mod, params=params)

In [33]:
import re
channels_re = re.compile('.*Tensor\[\(([\d]+), ([\d]+), [\d]+, [\d]+\).*padding.*Tensor\[\([\d]+, [\d]+, ([\d]+), ([\d]+)\).*')
cast_re = re.compile('cast.*Tensor\[\([\d]+, [\d]+, ([\d]+), ([\d]+)\).*')





In [34]:
print(mod.astext(show_meta_data=False))

#[version = "0.0.5"]
def @main(%input0: Tensor[(1, 3, 224, 224), float32]) -> Tensor[(1, 1000), float32] {
  %0 = strided_slice(%input0, begin=[0, 0, 0, 0], end=[1, 3, 224, 224], strides=[1, 1, 1, 1]) /* ty=Tensor[(1, 3, 224, 224), float32] */;
  %1 = take(%0, 0 /* ty=int32 */, axis=1, mode="wrap") /* ty=Tensor[(1, 224, 224), float32] */;
  %2 = expand_dims(%1, axis=1) /* ty=Tensor[(1, 1, 224, 224), float32] */;
  %3 = multiply(%2, 0.458f /* ty=float32 */) /* ty=Tensor[(1, 1, 224, 224), float32] */;
  %4 = add(%3, -0.03f /* ty=float32 */) /* ty=Tensor[(1, 1, 224, 224), float32] */;
  %5 = strided_slice(%input0, begin=[0, 0, 0, 0], end=[1, 3, 224, 224], strides=[1, 1, 1, 1]) /* ty=Tensor[(1, 3, 224, 224), float32] */;
  %6 = take(%5, 1 /* ty=int32 */, axis=1, mode="wrap") /* ty=Tensor[(1, 224, 224), float32] */;
  %7 = expand_dims(%6, axis=1) /* ty=Tensor[(1, 1, 224, 224), float32] */;
  %8 = multiply(%7, 0.448f /* ty=float32 */) /* ty=Tensor[(1, 1, 224, 224), float32] */;
  %9 = add(%8

In [26]:
mod_as_string = mod.astext(show_meta_data=False)
count=0
cast_line = ""
cast_line_idx = -1
final_workloads = []
for i, line in enumerate(mod_as_string.split('\n')):
    if "cast" in line and "int8" in line:
        cast_line = line
        cast_line_idx = i
    elif "max_pool2d" in line:
        print(line)
        #         match = re.search(channels_re, line)
        #         if match:
        #             if int(match.group(1)) % 16 == 0 and int(match.group(2)) % 16 == 0:
        #                 match_cast = re.search(cast_re, cast_line)
        #                 if match_cast:
        #                     wkl = workloads[count]
        #                     final_workloads.append(
        #                     'Workload({}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {})'.format(
        #                     1, match_cast.group(1), match_cast.group(2), wkl.in_filter, wkl.out_filter,
        #                     wkl.hkernel,wkl.wkernel, wkl.hpad, wkl.wpad, wkl.hstride, wkl.wstride))
        #                     count += 1
        

  %20 = nn.max_pool2d(%19, pool_size=[3, 3], strides=[2, 2], padding=[0, 0, 0, 0], ceil_mode=True) /* ty=Tensor[(1, 64, 56, 56), float32] */;
  %41 = nn.max_pool2d(%40, pool_size=[3, 3], strides=[2, 2], padding=[0, 0, 0, 0], ceil_mode=True) /* ty=Tensor[(1, 192, 28, 28), int8] */;
  %96 = nn.max_pool2d(%95, pool_size=[3, 3], padding=[1, 1, 1, 1], ceil_mode=True) /* ty=Tensor[(1, 192, 28, 28), int8] */;
  %164 = nn.max_pool2d(%163, pool_size=[3, 3], padding=[1, 1, 1, 1], ceil_mode=True) /* ty=Tensor[(1, 256, 28, 28), int8] */;
  %181 = nn.max_pool2d(%180, pool_size=[3, 3], strides=[2, 2], padding=[0, 0, 0, 0], ceil_mode=True) /* ty=Tensor[(1, 480, 14, 14), int8] */;
  %230 = nn.max_pool2d(%229, pool_size=[3, 3], padding=[1, 1, 1, 1], ceil_mode=True) /* ty=Tensor[(1, 480, 14, 14), int8] */;
  %296 = nn.max_pool2d(%295, pool_size=[3, 3], padding=[1, 1, 1, 1], ceil_mode=True) /* ty=Tensor[(1, 512, 14, 14), int8] */;
  %362 = nn.max_pool2d(%361, pool_size=[3, 3], padding=[1, 1, 1, 1], ceil_

In [18]:
print(mod_as_string)

#[version = "0.0.5"]
def @main(%input0: Tensor[(1, 3, 224, 224), float32]) -> Tensor[(1, 1000), float32] {
  %0 = nn.conv2d(%input0, meta[relay.Constant][0] /* ty=Tensor[(64, 3, 7, 7), float32] */, strides=[2, 2], padding=[3, 3, 3, 3], channels=64, kernel_size=[7, 7]) /* ty=Tensor[(1, 64, 112, 112), float32] */;
  %1 = add(%0, meta[relay.Constant][1] /* ty=Tensor[(64, 1, 1), float32] */) /* ty=Tensor[(1, 64, 112, 112), float32] */;
  %2 = nn.relu(%1) /* ty=Tensor[(1, 64, 112, 112), float32] */;
  %3 = nn.max_pool2d(%2, pool_size=[3, 3], strides=[2, 2], padding=[1, 1, 1, 1]) /* ty=Tensor[(1, 64, 56, 56), float32] */;
  %4 = annotation.stop_fusion(%3) /* ty=Tensor[(1, 64, 56, 56), float32] */;
  %5 = multiply(%4, 16f /* ty=float32 */) /* ty=Tensor[(1, 64, 56, 56), float32] */;
  %6 = round(%5) /* ty=Tensor[(1, 64, 56, 56), float32] */;
  %7 = clip(%6, a_min=-127f, a_max=127f) /* ty=Tensor[(1, 64, 56, 56), float32] */;
  %8 = cast(%7, dtype="int8") /* ty=Tensor[(1, 64, 56, 56), int8] */;


In [76]:
final_workloads = list(dict.fromkeys(final_workloads))
for i, wkl in enumerate(final_workloads):
    print('(\'workloads_{}\', {}),'.format(i, wkl))

('workloads_0', Workload(1, 112, 112, 32, 96, 1, 1, 0, 0, 2, 2)),
('workloads_1', Workload(1, 112, 112, 32, 96, 1, 1, 0, 0, 1, 1)),
('workloads_2', Workload(1, 112, 112, 96, 96, 3, 3, 1, 1, 2, 2)),
('workloads_3', Workload(1, 56, 56, 96, 96, 1, 1, 0, 0, 1, 1)),
('workloads_4', Workload(1, 56, 56, 96, 96, 3, 3, 1, 1, 1, 1)),
('workloads_5', Workload(1, 56, 56, 96, 192, 1, 1, 0, 0, 2, 2)),
('workloads_6', Workload(1, 56, 56, 96, 192, 1, 1, 0, 0, 1, 1)),
('workloads_7', Workload(1, 56, 56, 192, 192, 3, 3, 1, 1, 2, 2)),
('workloads_8', Workload(1, 28, 28, 192, 192, 1, 1, 0, 0, 1, 1)),
('workloads_9', Workload(1, 28, 28, 192, 192, 3, 3, 1, 1, 1, 1)),
('workloads_10', Workload(1, 28, 28, 192, 432, 1, 1, 0, 0, 2, 2)),
('workloads_11', Workload(1, 28, 28, 192, 432, 1, 1, 0, 0, 1, 1)),
('workloads_12', Workload(1, 28, 28, 432, 432, 3, 3, 1, 1, 2, 2)),
('workloads_13', Workload(1, 14, 14, 432, 432, 1, 1, 0, 0, 1, 1)),
('workloads_14', Workload(1, 14, 14, 432, 432, 3, 3, 1, 1, 1, 1)),
('workloads